In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

import re




import json
import os
from data_preprocess import DataTransformer
random_state = 12345

In [33]:


def features_interval(features, target, date1, date2):
    """
    Функция для выделения временных интервалов из таблиц признаков и целей
    на этом этапе отбрасываем колонку 'date'
    """
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    
    return features_interval, target_interval

In [34]:
def mae_day(y_true, y_pred):
    """
    Функция для вычисления метрики mae по дням из почасовых массивов данных
    """
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)

In [35]:
transformer = DataTransformer()

In [36]:
path = 'data/test_dataset.csv'
all_ds, open_test_begin, open_test_end = transformer.open_file(path)

начало закрытого теста: 2023-04-01 00:00:00     конец закрытого теста: 2023-08-01 00:00:00


In [37]:
all_ds = transformer.transform(all_ds)

AttributeError: 'NoneType' object has no attribute 'date'

In [ ]:
# применяем функцию добавления ВВП
all_ds = add_vvp2(all_ds)

In [ ]:
all_ds = add_true_weather(all_ds)

In [ ]:
# Итоговый набор колонок
train_ds.columns

In [ ]:
train_ds.head()

In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] #, 'temp_pred'] #, 'target_lag_336'] 

for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

In [ ]:
# Формируем набор датасетов для обучения и проверки

features = train_ds[feature_cols]
target = train_ds['target']

# для первичного подбора гиперпараметров будем обучать на 19-21 годах, валидировать по 2022
features_train, target_train = features_interval(features, target, '2019-01-01', '2022-01-01')
features_valid, target_valid = features_interval(features, target, '2022-01-01', '2023-01-01')

# отбор признаков будем производить, обучая на 19-22 и проверяя по первому кварталу 2023
# с дополнительным контролем на вариантах из первичного обучения
features_2022, target_2022 = features_interval(features, target, '2019-01-01', '2023-01-01')
features_2023, target_2023 = features_interval(features, target, '2023-01-01', open_test_begin)

# для проверки на тестовой выборке будем учиться на всем тренировочном датасете
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', open_test_begin)
features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)

In [ ]:
# формируем наборы данных по кварталам 2022 года, чтобы посмотреть по ним метрику отдельно
dates = ['2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01', '2023-01-01']
quarters = []
for i in range(4):
    f, t = features_interval(features, target, dates[i], dates[i+1])
    quarters.append({'features':f, 'target':t})

# Здесь заканчивается формирование датасетов и начинается обучение

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете

lgbm_model_all_train = lgb.LGBMRegressor(num_leaves=15, learning_rate=0.02, num_iterations=10000, 
                               feature_fraction=0.987, random_state=random_state, objective='regression_l1', n_jobs=-1)
lgbm_model_all_train.fit(features_all_train, target_all_train)

mae = mae_day(target_open_test, lgbm_model_all_train.predict(features_open_test))
print(f'mae_of_the_day - {mae}')